In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
import re

%matplotlib inline
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
DELTA_MEAN = 64
REVIEW_MEAN = 1041
REVIEW_MEDIAN = 904
    

df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')

# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
def review_time(row):
    match = re.findall('\d\d/\d\d/\d\d\d\d', str(row['Reviews']))
    fake = pd.to_datetime('10/25/2017', format='%m/%d/%Y')
    doomsday = pd.to_datetime('03/18/2020', format='%m/%d/%Y')
    if len(match) == 0:
        return [fake, fake, REVIEW_MEDIAN]
    elif len(match) == 1:
        return [pd.to_datetime(match[0], format='%m/%d/%Y'), fake, \
                (doomsday - pd.to_datetime(match[0], format='%m/%d/%Y')).days]
    t1 = pd.to_datetime(match[0],format='%m/%d/%Y')
    t2 = pd.to_datetime(match[1], format='%m/%d/%Y')
    if t1 > t2:
        t3 = doomsday - t1
    else:
        t3 = doomsday - t2
    return [t1, t2, t3.days]

def create_dict(df):
    cuisine_dict = {}
    cuisine_freq = {}
    idx = 0 
    for each in df['Cuisine Style']:
        for cuisine in each:
            if not cuisine in cuisine_dict:
                cuisine_dict[cuisine] = idx
                cuisine_freq[cuisine] = 1
                idx += 1
            else:
                cuisine_freq[cuisine] += 1
    return [cuisine_dict, cuisine_freq]

def dummy_venue(row):
    temp = [0,0,0,0,0,0,0]
    bar_venue = ['Bar', 'Pub', 'Wine Bar', 'Brew Pub']
    fast_venue = ['Pizza', 'Fast Food', 'Street Food']
    cafe_venue = ['Cafe', 'Gastropub', 'Diner']
    vegan_venue = ['Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options']
    meat_venue = ['Grill', 'Steakhouse', 'Barbecue']
    seafood_venue = ['Seafood', 'Sushi']
    for each in row['Cuisine Style']:
        if each == 'NaN':
            temp[0] = 1
        elif each in bar_venue:
            temp[1] = 1
        elif each in fast_venue:
            temp[2] = 1
        elif each in cafe_venue:
            temp[3] = 1
        elif each in vegan_venue:
            temp[4] = 1
        elif each in meat_venue:
            temp[5] = 1
        else:
            temp[6] = 1
    return pd.Series(temp)

In [ ]:
pd.isna(data['City']).value_counts()

In [ ]:
def preproc_data(df_input):
    
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA'], axis = 1, inplace=True)
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...

    # тут ваш код по обработке NAN
    # ....
    
    df_output['Cuisine Style_isNAN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Price Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Reviews_isNAN'] = pd.isna(df_output['Reviews']).astype('uint8')

    # Далее заполняем пропуски
    
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Price Range'].fillna('$$ - $$$', inplace=True)
    df_output['Cuisine Style'].fillna('NaN', inplace=True)
    df_output['Reviews'].fillna('NaN', inplace=True)
    
    print(df_output.info())
    # ################### 3. Encoding ############################################################## 
    
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    #Кодирование городов методом Label Encoding
    #le = LabelEncoder()
    #df_output['City Code'] = le.fit_transform(df_input['City'])
    
    price_range_dict = {
    '$': 1,
    '$$ - $$$': 2,
    '$$$$': 3
    }
    df_output['Price Range'] = df_output['Price Range'].map(price_range_dict)
    
    print(df_output.info())
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код на генерацию новых фичей
    # ....
    df_output['Ranking_norm'] = df_input['Ranking'] / df_input['City'].map(df_input.groupby(['City'])['Ranking'].max())
    
    # создание признака "население городов"
    population = {'Paris': 2190327, 'Stockholm': 961609, 'London': 8908081, 'Berlin': 3644826, 'Munich': 1456039, 
                  'Oporto': 237591,
                  'Milan': 1378689,'Bratislava': 432864, 'Vienna': 1821582, 'Rome': 4355725, 
                  'Barcelona': 1620343, 'Madrid': 3223334,
                  'Dublin': 1173179,'Brussels': 179277, 'Zurich': 4+28737, 'Warsaw': 1758143, 
                  'Budapest': 1752286, 'Copenhagen': 615993,
                  'Amsterdam': 857713,'Lyon': 506615, 'Hamburg': 1841179,'Lisbon': 505526, 
                  'Prague': 1301132, 'Oslo': 673469,
                  'Helsinki': 643272,'Edinburgh': 488100,'Geneva': 200548, 'Ljubljana': 284355,
                  'Athens': 664046, 'Luxembourg': 115227,
                  'Krakow': 769498}
    
    df_output['Population'] = df_input['City'].map(population)
    df_output['review_norm'] = df_output['Number of Reviews']/df_output['Population']
    
    #Количеcтво ресторанов на город, берем по максимальному значению Ranking
    df_output['count_city_venue'] = df_input['City'].map(df_input.groupby(['City'])['Ranking'].max().to_dict())
    
    df_output['Pop_Rest'] = df_input['City'].map(population) / df_output['count_city_venue']
    df_output['Rev_Rest'] = df_output['Number of Reviews'] / df_output['Pop_Rest']
    
    df_temp = df_output.apply(review_time, axis=1, result_type='expand')
    fake = pd.to_datetime('10/25/2017',  format='%m/%d/%Y')
    

    df_output['delta'] = df_temp.apply(lambda x: abs((x[0]-x[1])).\
                                       days if not x[0] == fake or not x[1] == fake else DELTA_MEAN, axis=1)
    
    df_output['is_good'] = df_input['Reviews'].apply(\
    lambda x: 0 if pd.isna(re.search("Good|good|excellent|Excellent|awesome|Awesome|Best|best|Nice|nice", str(x))) else 1)
    
    #City is capital?
    
    capitals = ['London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 
            'Lisbon', 'Vienna', 'Amsterdam', 'Brussels', 'Stockholm', 
            'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens', 
            'Oslo', 'Helsinki', 'Bratislava', 'Luxembourg', 'Ljubljana','Edinburgh']
    
    df_output['City Cap'] = df_input['City'].apply(lambda x: 1 if x in capitals else 0)
    
    #df_output['rest_id'] = df_input['Restaurant_id'].map(lambda x: int(x[3:]))
    df_output['id_ta'] = df_input['ID_TA'].map(lambda x: int(x[1:]))
    
  
    #cuisine 
     
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: re.findall('\w+\s*\w+\s*\w+', str(x)))
    [cuisine_dict, cuisine_freq] = create_dict(df_output)
    df_output['cuisine count'] = df_output['Cuisine Style'].map(lambda x: len(x))
    df_output['max_cuisine_idx'] = df_output.apply(lambda row: max([cuisine_dict[x] for x in row['Cuisine Style']]), axis=1)
    
    dummy_cuisine = df_output.apply(dummy_venue, axis=1)
    dummy_cuisine.columns = ['boom', 'bar', 'cafe', 'fastfood', 'vegan', 'meat', 'national']
    
    df_output = df_output.join(dummy_cuisine)
    
    ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    #Удаляем столбцы в поисках оптимальных сочетаний
    to_drop = ['City_nan']
    df_output.drop(to_drop, axis = 1, inplace=True)
    
    return df_output

In [7]:
df_preproc = preproc_data(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   City                     50000 non-null  object 
 1   Cuisine Style            50000 non-null  object 
 2   Ranking                  50000 non-null  float64
 3   Price Range              50000 non-null  object 
 4   Number of Reviews        50000 non-null  float64
 5   Reviews                  50000 non-null  object 
 6   URL_TA                   50000 non-null  object 
 7   sample                   50000 non-null  int64  
 8   Rating                   50000 non-null  float64
 9   Cuisine Style_isNAN      50000 non-null  uint8  
 10  Price Range_isNAN        50000 non-null  uint8  
 11  Number_of_Reviews_isNAN  50000 non-null  uint8  
 12  Reviews_isNAN            50000 non-null  uint8  
dtypes: float64(3), int64(1), object(5), uint8(4)
memory usage: 3.6+ MB
None
<cla

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)



# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)



# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)


# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)



# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))



# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')